未完成

url : https://www.youtube.com/watch?v=ijvCaT_DXV4

In [ ]:
# !apt-get remove mecab libmecab-dev mecab-ipadic-utf8
!apt-get install -y mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libmecab2 mecab-ipadic mecab-utils
The following NEW packages will be installed:
  libmecab-dev libmecab2 mecab mecab-ipadic mecab-ipadic-utf8 mecab-utils
0 upgraded, 6 newly installed, 0 to remove and 30 not upgraded.
Need to get 7,367 kB of archives.
After this operation, 59.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libmecab2 amd64 0.996-14build9 [199 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libmecab-dev amd64 0.996-14build9 [306 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 mecab-utils amd64 0.996-14build9 [4,850 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 mecab-ipadic all 2.7.0-20070801+main-3 [6,718 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 mecab amd64 0.996-14build9 [136 kB]
Get:6 http://archive.ubuntu.co

In [ ]:
# https://github.com/SamuraiT/mecab-python3#common-issues
!pip install unidic-lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658817 sha256=ff9b2be8aa1847ba605e2838ee0fc10fdcc41d3a3431bf99ebaaae1a58a4017b
  Stored in directory: /root/.cache/pip/wheels/89/e8/68/f9ac36b8cc6c8b3c96888cd57434abed96595d444f42243853
Successfully built unidic-lite


In [ ]:
### 形態素解析
import MeCab

#text = "私はお昼休みに美味しいカレーライスを食べました。"

m = MeCab.Tagger()

def mecab_sep(text):
    node = m.parseToNode(text)

    words_list = []

    while node:
        if node.feature.split(",")[0] == "動詞":
            words_list.append(node.feature.split(",")[6])
        elif node.feature.split(",")[0] == "形容詞":
            words_list.append(node.feature.split(",")[6])
        else:
            words_list.append(node.surface)

        node = node.next

    return words_list[1:-1]

In [ ]:
# https://yag-ays.github.io/project/pretrained_doc2vec_wikipedia/
# versionを変えたらいけるらしい

from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec.load("jawiki.doc2vec.dbow300d/jawiki.doc2vec.dbow300d.model")

FileNotFoundError: [Errno 2] No such file or directory: 'jawiki.doc2vec.dbow300d/jawiki.doc2vec.dbow300d.model'

In [ ]:
def calc_vecs_d2v(docs):
    vecs = []
    for d in docs:
        vecs.append(model.infer_vector(mecab_sep(d)))

    return vecs

In [ ]:
### Bag-of-words / tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer

def calc_vecs(docs):
    vectorizer = TfidfVectorizer(analyzer=mecab_sep)
    vecs = vectorizer.fit_transform(docs)
    return vecs.toarray()


In [ ]:
### コサイン類似度
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

input_doc = "君はネコが好きです。"

target_docs_df = pd.read_csv("/content/nlp_datascience_cram_school_2.csv", encoding='shift_jis')
target_docs = target_docs_df["文章リスト"].tolist()

all_docs = [input_doc] + target_docs
all_docs_vecs = calc_vecs(all_docs)

similarity =cosine_similarity([all_docs_vecs[0]],all_docs_vecs[1:])

target_docs_df["類似度"] = similarity[0]

In [ ]:
target_docs_df.sort_values("類似度",ascending=False)

,文章リスト,類似度
0,私は犬が好きです。,0.438497
4,あなたは猫が好きです。,0.388658
2,私は犬がとても好きです。,0.354345
3,あなたは犬と猫が好きです。,0.310741
1,私は犬が嫌いです。,0.278822


In [ ]:
target_docs_df.sort_values("類似度",ascending=False)

,文章リスト,類似度
4,あなたは猫が好きです。,0.924543
3,あなたは犬と猫が好きです。,0.919218
0,私は犬が好きです。,0.907596
1,私は犬が嫌いです。,0.903666
2,私は犬がとても好きです。,0.902655


In [ ]:
import glob
target_docs=[]

for x in glob.glob("text/*/*.txt"):
    text = ""
    with open(x) as f:
        next(f)
        next(f)
        for line in f:
            text = text + line
    f.close()
    target_docs.append(text)


In [ ]:
all_docs_vecs = calc_vecs_d2v(target_docs)

In [ ]:
### コサイン類似度
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

input_doc = "お前らどんだけ起きてんだよ！"

similarity =cosine_similarity(calc_vecs_d2v([input_doc]),all_docs_vecs)

target_docs_df = pd.DataFrame(target_docs,columns=["文章リスト"])
target_docs_df["類似度"] = similarity[0]

In [ ]:
target_docs_df.sort_values("類似度",ascending=False).head(5)["文章リスト"].tolist()[1]

'ツイッターでストレス発散？\u3000ローラが老人の弁当のおかずを聞かれ「もちーo(^u^)o」【話題】\nツイッターが人気のタレントと言えば思い出すのがローラ。そんな彼女がまた仰天発言を行った。\nファンからの「ローラ明日おばあちゃんのゲートボール大会あるんだけどおかず何がいいと思う？」というツイートに対するローラの返事は「もちーo(^u^)o」だったのだ。\n\nおばあちゃんにお餅！\u3000と思ってしまったが、ネット上でも「おばあちゃん危ない」「見た瞬間爆笑した」などの声があがっている。ローラの返事を面白がっている人が多いのだが、中には「ツイッターでストレス発散してるのかな」などの書き込みもあった。\n\nどうも気になるローラのツイッター。目が離せない。\n\n＠ローラツイッター\n'